In [7]:
import configparser
import os
import pickle

import numpy as np
import sktensor

import logging
logging.basicConfig(level=logging.INFO, 
                    format='%(msecs)d %(levelname)-8s [%(lineno)d] %(message)s')

In [2]:
config = configparser.ConfigParser()                                            
config.read('config.ini')                                                       

['config.ini']

# Tucker

In [2]:
from eval_tensor import VerbTensorEvaluator

In [3]:
evalor = VerbTensorEvaluator()

In [4]:
evalor.load_embeddings()

In [9]:
tensor = evalor.decomped_tns

In [102]:
tensor.shape

(4746, 4757, 3939)

In [53]:
tensor.core.shape

(128, 128, 128)

In [60]:
tensor.core.reshape(-1)


array([-5.44164749e+02, -7.55266333e+02,  9.47788670e+01, ...,
        6.58042067e-02, -3.97073285e-02, -6.32675680e-02])

In [101]:
for s_i, v_i, o_i in [np.unravel_index(i, tensor.core.shape) 
                      for i in list(reversed((tensor.core.reshape(-1)).argsort()))[:10]]:
    print([evalor.index['nsubj'].inv[i] for i in list(reversed(tensor.factors[0][s_i].argsort()))[:10]])
    print([evalor.index['ROOT'].inv[i] for i in list(reversed(tensor.factors[1][v_i].argsort()))[:10]])
    print([evalor.index['dobj'].inv[i] for i in list(reversed(tensor.factors[2][o_i].argsort()))[:10]])
    print()

['that', 'you', 'I', 'it', 'this', 'people', 'one', 'she', 'time', '-lrb-']
['make', 'go', 'have', 'get', 'take', 'look', 'see', 'feel', 'work', 'use']
['it', 'you', 'they', 'I', 'he', 'answer', '-rrb-', 'this', 'change', 'that']

['we', 'he', 'that', 'people', 'what', 'they', 'i', 'who', 'which', 'way']
['have', 'say', 'go', 'look', 'start', 'feel', 'make', 'think', 'ask', 'need']
['what', 'use', 'you', 'we', 'more', 'thing', 'this', '-rrb-', 'game', 'post']

['I', 'they', 'she', 'one', 'thing', 'that', 'what', 'who', 'people', 'it']
['have', 'say', 'go', 'look', 'start', 'feel', 'make', 'think', 'ask', 'need']
['it', 'you', 'they', 'I', 'he', 'answer', '-rrb-', 'this', 'change', 'that']

['that', 'you', 'I', 'it', 'this', 'people', 'one', 'she', 'time', '-lrb-']
['have', 'say', 'go', 'look', 'start', 'feel', 'make', 'think', 'ask', 'need']
['it', 'you', 'they', 'I', 'he', 'answer', '-rrb-', 'this', 'change', 'that']

['we', 'he', 'that', 'people', 'what', 'they', 'i', 'who', 'which',

In [97]:
    
    print([evalor.index['ROOT'].inv[i] for i in reversed(tensor.factors[1][v_i].argsort())]
    print([evalor.index['dobj'].inv[i] for i in reversed(tensor.factors[1][o_i].argsort())]
    print()

In [67]:
np.unravel_index(tensor.core.argmax(), tensor.core.shape)

(1, 0, 0)

In [73]:
for latent in range(16):
    print([evalor.index['ROOT'].inv[i] for i in (-tensor.factors[1][latent]).argsort()][:10])

['make', 'go', 'have', 'get', 'take', 'look', 'see', 'feel', 'work', 'use']
['have', 'say', 'go', 'look', 'start', 'feel', 'make', 'think', 'ask', 'need']
['think', '-rrb-', 'want', 'have', 'do', 'look', 'love', 'tell', 'find', 'change']
['work', 'start', 'give', 'find', 'see', 'show', 'need', 'take', 'have', 'love']
['come', 'take', '-rrb-', 'use', 'have', 'work', 'know', 'look', 'say', 'make']
['put', 'come', 'tell', 'show', 'buy', 'write', 'meet', 'find', 'want', 'enjoy']
['want', '-rrb-', 'have', 'help', 'use', 'get', 'give', 'know', 'try', 'like']
['return', 'point', 'feel', 'remember', 'seem', 'way', 'increase', 'thank', 'set', 'walk']
['use', 'think', 'get', 'have', 'say', 'go', 'see', 'help', 'come', 'mean']
['thank', 'click', 'available', 'follow', 'agree', 'support', 'like', 'set', 'put', 'pay']
['know', 'try', 'get', 'have', 'go', 'give', 'seem', 'think', 'tell', 'love']
['improve', 'open', 'can', 'stand', 'wish', 'guess', 'plan', 'bring', 'keep', 'seem']
['see', '-rrb-', 'n

# Triples by `lmbda`

In [5]:
tensor_dir = config['DEFAULT']['ProjectDirectory']+'tensor/'
def write_latent_dims(weight, rank=256, cutoff=100):
    modes = ['nsubj', 'ROOT', 'dobj']
    sparse_tensor, index = pickle.load(open(os.path.join(tensor_dir, 'sparstensr_{}_{}.pkl').format(
        weight, cutoff), mode='rb'))
    ktensor, _, _, _ = pickle.load(open(os.path.join(tensor_dir, 'ktensor_{}_{}_{}.pkl').format(
        weight, cutoff, rank), 
        mode='rb'))
    i_word = [{i: w for w, i in index[j].items()} for j in modes]
    vocab = [np.array([i_word[j][i] for i in range(len(i_word[j]))]) for j in range(len(i_word))]
    for k, j in enumerate(
            #np.argsort(-ktensor.lmbda)[:4]):#
            range(ktensor.rank)):
        print('\n{} {}'.format(k, j))
        for i in range(len(ktensor.U)):
            ids = np.argsort(ktensor.U[i][:,j])[:7]
            print(vocab[i][ids])
            #print(['{:.1g}'.format(coord) for coord in ktensor.U[i][ids,j]])

In [6]:
write_latent_dims('log_freq')#, 64, 5)


0 0
['system' 'company' 'program' 'country' 'state' 'application' 'game']
['-lrb-' 'note' 'mention' 'ensure' 'select' 'name' 'replace']
['' 'value' 'the' 'effect' '%' 'those' '-rrb-']

1 1
['item' 'record' 'image' '-lsb-' 'area' 'effect' 'room']
['get' 'sell' 'lose' 'create' 'send' 'carry' 'save']
['hundred' 'eye' 'material' 'hand' 'match' 'arm' 'pressure']

2 2
['it' 'he' 'they' 'you' 'I' 'this' 'she']
['have' 'take' 'do' 'give' 'provide' 'watch' 'contain']
['' 'what' 'way' 'that' 'you' 'it' 'I']

3 3
['she' 'I' 'this' '-rrb-' 'time' 'thing' 'party']
['have' 'take' 'need' 'get' 'offer' 'see' 'provide']
['' 'it' 'number' 'what' 'information' 'use' 'that']

4 4
['everyone' 'anyone' 'person' 'company' 'woman' 'teacher' 'somebody']
['include' 'share' 'reduce' 'require' 'experience' 'contain' 'manage']
['' 'ourselves' 'goal' 'himself' 'opinion' 'quality' 'level']

5 5
['' 'I' 'you' 'he' 'we' 'they' 'she']
['be' 'make' 'find' 'require' 'announce' 'cause' 'spend']
['' 'he' 'what' 'I' 'way' 

['Smith' 'Jones' 'Brown' 'Jackson' 'Johnson' 'Williams' 'King']
['be' 'do' 'provide' 'use' 'keep' 'lead' 'continue']
['' 'what' 'it' 'effect' 'use' 'work' 'impact']

83 83
['' 'he' 'you' 'they' 'it' 'that' 'we']
['see' 'leave' 'get' 'have' 'show' 'use' 'make']
['I' 'you' 'we' 'they' 'what' 'he' 'she']

84 84
['' 'they' 'we' 'he' 'who' 'that' 'you']
['add' 'show' 'serve' 'keep' 'run' 'turn' 'prevent']
['experience' 'work' 'service' '-rrb-' 'program' 'game' 'it']

85 85
['' 'I' 'you' 'trip' 'journey' 'travels' 'work']
['take' 'use' 'build' 'support' 'show' 'have' 'require']
['' 'place' 'what' 'it' 'change' 'team' 'work']

86 86
['thing' 'situation' 'problem' 'world' 'machine' 'item' 'size']
['need' 'put' 'take' 'pay' 'play' 'get' 'feel']
['' 'it' 'what' 'information' 'attention' 'effect' '$']

87 87
['' 'that' 'which' 'who' 'it' 'he' 'I']
['have' 'be' 'take' 'find' 'see' 'contain' 'win']
['something' 'thing' 'anything' 'place' 'side' 'face' 'much']

88 88
['who' 'we' 'he' 'she' 'which' '

['it' 'that' 'which' 'this' 'they' 'we' '-rrb-']
['cover' 'contain' 'prevent' 'put' 'join' 'serve' 'say']
['time' '-lrb-' '-rrb-' 'number' 'it' 'rate' 'value']

166 166
['what' 'time' 'kid' 'girl' 'number' 'boy' 'life']
['run' 'have' 'go' 'keep' 'lead' 'work' 'cut']
['way' 'that' 'they' 'which' 'call' 'lot' 'request']

167 167
['' 'I' 'you' 'they' 'we' 'he' 'who']
['find' 'show' 'see' 'pay' 'want' 'set' 'use']
['make' 'campaign' 'be' 'get' 'position' '``' 'fund']

168 168
['company' 'team' 'these' 'people' 'those' 'player' 'man']
['do' 'have' 'make' 'tell' 'help' 'keep' 'follow']
['' 'it' 'you' 'I' 'that' 'this' 'we']

169 169
['' 'he' 'they' 'I' 'you' 'we' 'who']
['find' 'turn' 'give' 'kill' 'put' 'ask' 'receive']
['that' 'it' 'thing' 'this' 'which' 'team' 'business']

170 170
['who' 'people' 'she' 'they' 'child' 'everyone' 'family']
['have' 'use' 'love' 'want' 'support' 'know' 'need']
['' 'number' 'set' 'name' 'link' 'point' 'review']

171 171
['I' 'we' 'you' 'they' 'he' 'who' 'it']


['' 'it' 'what' 'effect' 'view' 'impact' 'role']

248 248
['' 'we' 'you' 'I' 'he' 'they' 'who']
['say' 'be' 'do' '-rrb-' 'think' 'add' 'tell']
['place' 'time' 'one' 'we' 'which' 'they' 'kind']

249 249
['we' 'you' 'they' 'he' 'who' 'she' 'I']
['share' 'put' 'see' 'create' 'present' 'offer' 'take']
['one' 'letter' 'call' 'people' 'bill' 'building' 'text']

250 250
['' 'I' 'what' 'feature' 'mother' 'page' 'woman']
['make' 'do' 'develop' 'use' 'buy' 'set' 'see']
['' 'information' 'system' '$' 'amount' 'number' 'program']

251 251
['' 'that' 'it' 'they' 'who' 'we' 'name']
['need' 'use' 'find' 'want' 'lose' 'provide' 'bring']
['' 'it' 'what' '$' 'effect' 'that' 'information']

252 252
['you' 'I' 'we' 'user' 'name' 'they' 'device']
['say' '-rrb-' 'offer' 'be' 'fall' 'fit' 'come']
['they' 'which' 'one' 'something' 'thing' 'anything' 'yourself']

253 253
['' 'this' 'what' 'name' 'report' 'area' 'argument']
['need' 'find' 'want' 'make' 'do' 'pay' 'say']
['' 'you' 'effect' 'that' 'I' '$' '-lrb-'

In [4]:
write_latent_dims('pmi', 64, 5)


0 50
['lapan' 'nc-noting' 'deshchytsia' 'markeson' 'mn-marveling' 'nikonov'
 'araghchi']
['feel' 'seem' 'be' 'sound' 'work' 'go' 'build']
['reporter' 'place' 'briefing' 'land' 'same' 'farm' 'leg']

1 61
['share' 'paper' 'chapter' 'nikkei' 'article' 'shares' 'god']
['prevent' 'allow' 'cause' 'enable' 'describe' 'explore' 'cost']
['user' 'i' 'you' 'we' 'rule' 'they' 'people']

2 22
['scorer' 'larkin' 'malone' 'individual' 'americans' 'jack' 'russia']
['start' 'enter' 'open' 'lose' 'leave' 'finish' 'play']
['game' 'we' 'i' 'advantage' 'record' 'rate' 'population']

3 24
['suite' 'hotel' 'accommodation' 'disruption' 'variability' 'imbalance'
 'inability']
['provide' 'be' 'go' 'set' 'include' 'seem' 'visit']
['effect' 'game' 'impact' 'consequence' 'season' 'implication' 'place']


In [6]:
write_latent_dims('dice_sali', 32, 5)


0 20
['rumor' 'legend' 'version' 'what' 'q.' 'rumour' 'luck']
['have' 'hold' 'open' 'hit' 'feature' 'reach' 'be']
['office' 'member' 'doctor' 'eye' 'head' 'arm' 'shop']

1 14
['fig.' 'journey' 'drawback' 'kde' 'travels' 'orbit' 'bought']
['take' 'hold' 'teach' 'experience' 'pay' 'learn' 'ask']
['attention' 'question' 'photo' 'lesson' 'tax' 'letter' 'message']

2 19
['room' 'it' 'each' 'property' 'unit' 'model' 'area']
['love' 'like' 'see' 'notice' 'enjoy' 'prefer' 'know']
['effect' 'it' 'room' 'impact' 'feature' 'area' 'potential']

3 12
['bedroom' 'q.' 'villa' '´' 'accommodation' 'condo' 'chart']
['make' 'raise' 'pay' 'offer' 'say' 'ask' 'give']
['question' 'problem' 'chance' 'hand' 'concern' 'idea' 'head']


|rank|$\lambda$|e.g.|
|----|---------|----|
|16|1|topic/example (2) provide/offer (2) assistance/guidance|
|32|1|Android (3)  win degree (1)|
|64|1|I describe (1) game/story (2)|